In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\pacho\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pyspark'

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (
            SparkSession
                .builder
                .appName("RDDApp")
                .master("local[4]")
                .getOrCreate()
        )

24/04/27 10:38:30 WARN Utils: Your hostname, emi-All-Series resolves to a loopback address: 127.0.1.1; using 192.168.100.35 instead (on interface enp3s0)
24/04/27 10:38:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/27 10:38:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/27 10:38:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark

### Create RDD - Option 1: Parallelize

In [3]:
# Create a variable for SparkContext

sc = spark.sparkContext

In [4]:
# Create RDD using parallelize

numbersRdd = sc.parallelize( [1, 2, 3, 4, 5] )

In [5]:
# Check RDD partitions

numbersRdd.getNumPartitions()

4

In [6]:
# Get result from RDD

output = numbersRdd.collect()

print(output)

[1, 2, 3, 4, 5]


In [7]:
# Get any 2 records from RDD

numbersRdd.take(2)

[1, 2]

In [11]:
numbersRdd.take(6)

[1, 2, 3, 4, 5]

In [12]:
# Get first record from RDD
# first() returns an element, while take(1) returns an array with one element

numbersRdd.first()

1

In [13]:
# Create RDD with complex types

employeesRdd = sc.parallelize(
                                [
                                    [ 1, "Neha",  10000 ],
                                    [ 2, "Steve", 20000 ],
                                    [ 3, "Kari",  30000 ],
                                    [ 4, "Ivan",  40000 ],
                                    [ 5, "Mohit", 50000 ]
                                ]
                              )

In [14]:
# Get first record from RDD

employeesRdd.first()

[1, 'Neha', 10000]

### Create RDD - Option 2: Read a File

In [43]:
# Read TaxiZones.csv file and create RDD

taxiZonesRdd = sc.textFile("datos/TaxiZones.csv")

In [44]:
taxiZonesRdd

datos/TaxiZones.csv MapPartitionsRDD[38] at textFile at NativeMethodAccessorImpl.java:0

In [45]:
# Get 10 records

taxiZonesRdd.take(10)

['1,EWR,Newark Airport,EWR',
 '2,Queens,Jamaica Bay,Boro Zone',
 '3,Bronx,Allerton/Pelham Gardens,Boro Zone',
 '4,Manhattan,Alphabet City,Yellow Zone',
 '5,Staten Island,Arden Heights,Boro Zone',
 '6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone',
 '7,Queens,Astoria,Boro Zone',
 '8,Queens,Astoria Park,Boro Zone',
 '9,Queens,Auburndale,Boro Zone',
 '10,Queens,Baisley Park,Boro Zone']

In [46]:
# Check RDD partitions

taxiZonesRdd.getNumPartitions()

2

In [47]:
# Read TaxiZones.csv file and create RDD (with 4 partitions)

taxiZonesRdd = sc.textFile("datos/TaxiZones.csv", 4)

taxiZonesRdd.getNumPartitions()

4

### Create RDD - Option 3: Convert another RDD

In [48]:
# Apply map operation to split each record by comma, and get array of items

taxiZonesWithColsRdd = (
                            taxiZonesRdd
    
                                .map( lambda zone: zone.split(",") )
                       )

taxiZonesWithColsRdd.take(5)

[['1', 'EWR', 'Newark Airport', 'EWR'],
 ['2', 'Queens', 'Jamaica Bay', 'Boro Zone'],
 ['3', 'Bronx', 'Allerton/Pelham Gardens', 'Boro Zone'],
 ['4', 'Manhattan', 'Alphabet City', 'Yellow Zone'],
 ['5', 'Staten Island', 'Arden Heights', 'Boro Zone']]

In [49]:
# Apply filter operation
# Get data where Borough is 'Manhattan' and Zone begins with 'central'

filteredZonesRdd = (
                        taxiZonesWithColsRdd
    
                            .filter( lambda zoneRow: zoneRow[1] == "Manhattan" 

                                                 and zoneRow[2].lower().startswith("central")
                                   )
                   )

filteredZonesRdd.take(5)

[['41', 'Manhattan', 'Central Harlem', 'Boro Zone'],
 ['42', 'Manhattan', 'Central Harlem North', 'Boro Zone'],
 ['43', 'Manhattan', 'Central Park', 'Yellow Zone']]

In [50]:
# Get list of Zones with even LocationId

evenZoneIds = (
                    taxiZonesRdd
    
                        .filter( lambda zoneRow: int(zoneRow[0]) % 2 == 0 )
              )

evenZoneIds.take(5)

['2,Queens,Jamaica Bay,Boro Zone',
 '4,Manhattan,Alphabet City,Yellow Zone',
 '6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone',
 '8,Queens,Astoria Park,Boro Zone',
 '20,Bronx,Belmont,Boro Zone']

### Pair RDDs

In [51]:
import math

numbersRDD = sc.parallelize( [2, 3, 4, 5, 6] )

# Create Pair RDD
numsWithSquareRootRdd = (
                            numbersRDD
    
                                .map(lambda num: (
                                                    num,
                                                    math.sqrt(num)
                                                 )
                                    )
                        )

numsWithSquareRootRdd.collect()

[(2, 1.4142135623730951),
 (3, 1.7320508075688772),
 (4, 2.0),
 (5, 2.23606797749979),
 (6, 2.449489742783178)]

### Exercise 1

Use taxiZonesWithColsRdd and create a Pair RDD (LocationId as Key, Zone Details as Value)

In [52]:
# Exercise 1 Answer

taxiZonesPairRdd = (
                        taxiZonesWithColsRdd
    
                            .map(lambda zoneRow: ( 
                                                    zoneRow[0],
                                                    zoneRow
                                                 )
                                )
                   )

taxiZonesPairRdd.take(10)

[('1', ['1', 'EWR', 'Newark Airport', 'EWR']),
 ('2', ['2', 'Queens', 'Jamaica Bay', 'Boro Zone']),
 ('3', ['3', 'Bronx', 'Allerton/Pelham Gardens', 'Boro Zone']),
 ('4', ['4', 'Manhattan', 'Alphabet City', 'Yellow Zone']),
 ('5', ['5', 'Staten Island', 'Arden Heights', 'Boro Zone']),
 ('6', ['6', 'Staten Island', 'Arrochar/Fort Wadsworth', 'Boro Zone']),
 ('7', ['7', 'Queens', 'Astoria', 'Boro Zone']),
 ('8', ['8', 'Queens', 'Astoria Park', 'Boro Zone']),
 ('9', ['9', 'Queens', 'Auburndale', 'Boro Zone']),
 ('10', ['10', 'Queens', 'Baisley Park', 'Boro Zone'])]

### Calculate count of records for each Borough

In [53]:
# Create Pair RDD (locationId as Key, Value as 1)

taxiZonesPairRdd = (
                        taxiZonesWithColsRdd
    
                            .map(lambda zoneRow: ( 
                                                    zoneRow[1],   # Key   - Borough
                                                    1             # Value - 1
                                                 )
                                )
                   )

taxiZonesPairRdd.take(10)

[('EWR', 1),
 ('Queens', 1),
 ('Bronx', 1),
 ('Manhattan', 1),
 ('Staten Island', 1),
 ('Staten Island', 1),
 ('Queens', 1),
 ('Queens', 1),
 ('Queens', 1),
 ('Queens', 1)]

In [54]:
# Calculate count of records for each Borough

boroughCountRdd = (
                        taxiZonesPairRdd
    
                            .reduceByKey( lambda value1, value2: value1 + value2 )
                  )

boroughCountRdd.collect()

[('Bronx', 43),
 ('Staten Island', 20),
 ('EWR', 1),
 ('Manhattan', 69),
 ('Brooklyn', 61),
 ('Unknown', 2),
 ('Queens', 69)]

In [55]:
# Sort the data by Key

(
    taxiZonesPairRdd
    
        .sortByKey()
    
        .collect()
)

[('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Bronx', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn', 1),
 ('Brooklyn',

In [56]:
# Only return the keys

(
    taxiZonesPairRdd
    
        .keys()
    
        .take(10)
)

['EWR',
 'Queens',
 'Bronx',
 'Manhattan',
 'Staten Island',
 'Staten Island',
 'Queens',
 'Queens',
 'Queens',
 'Queens']

In [57]:
# Only return the values

(
    taxiZonesPairRdd
    
        .values()
    
        .take(10)
)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [58]:
# Return distinct pairs

(
    taxiZonesPairRdd
    
        .distinct()
    
        .collect()
)

[('Bronx', 1),
 ('Staten Island', 1),
 ('Queens', 1),
 ('EWR', 1),
 ('Manhattan', 1),
 ('Brooklyn', 1),
 ('Unknown', 1)]